## imports

In [1]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

2025-05-23 15:04:58.810012: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-23 15:04:58.920579: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-23 15:05:03.018023: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/

In [2]:
import pickle
import sys
sys.path.append('../')
sys.path.append('../../../../')

## constants

In [3]:
data_for_training_dir_path = source_dataset_dir_path = '/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training'
source_dataset_dir_path = f'{data_for_training_dir_path}/03_04_with_pesto'
target_dataset_dir_path = f'{data_for_training_dir_path}/03_04_with_pesto_and_coord'

MAX_NUMBER_OF_COMPONENTS = 10

## utils

In [72]:
# copied from data_development_utils.py
# modified a bit
# TODO: the main risk here: sorted might change order from original creation 

def extract_protein_data(proteins, max_number_of_components):
    residue_indexes = []
    for protein in proteins:
        # Sort components by average_ubiq in descending order and take the top 10
        top_components = sorted(protein.connected_components_tuples, key=lambda x: x[1]['average_scanNet_ubiq'], reverse=True)[
                         :max_number_of_components]
        residue_indexes.append([component[2] for component in top_components])
    return residue_indexes

## load data from source

for fold_index in range(5) <br>
for set_name in ['train', 'validation', 'test'] <br>

Currently folds_training_dicts[fold_index] of set_name, contains:
- sizes (will be broadcasted)
- num_patches (will be broadcasted)
- components (the features matrix, of shape [protein_amount, 10, 9] -> 10=patches_amount, 9=2 for scannet, 5 for pesto, 1 for plddt, 1 for size)
- uniprots
- labels

We want to add one more matrix (again, per fold_index, per set_name). <br>
This matrix will be of shape [protein_amount, 10, 3] -> x, y, z coordiantes. <br>
x, y, z coordinates will be the average of the ca coords in the patch. <br>
the new key will be:

- coordinates

In [4]:
with open(f'{source_dataset_dir_path}/folds_training_dicts.pkl', 'rb') as f:
    folds_training_dicts = pickle.load(f)

2025-05-23 15:05:14.816902: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# TODO: itterate in for loop
fold_index = 0
fold = folds_training_dicts[fold_index]
[x for x in fold.keys() if 'train' in x]

['sizes_train',
 'components_train',
 'num_patches_train',
 'uniprots_train',
 'labels_train']

In [7]:
with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/patches_dicts/03_04_with_pesto/merged_protein_objects.pkl', 'rb') as f:
# with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/patches_dicts/03_04_with_pesto/proteinObjectsWithEvoluion0', 'rb') as f:
    merged_protein_objects = pickle.load(f)

In [18]:
set_name = 'train'  # TODO: iterate in for loop
uniprots = set(fold[f'uniprots_{set_name}'])
len(uniprots), list(uniprots)[0]

(76632, 'Q9XV59')

In [24]:
fold[f'components_{set_name}'].shape

TensorShape([76632, 10, 9])

In [75]:
data_set_protein_objects = {k: v for k, v in merged_protein_objects.items() if k in uniprots}
data_set_res_indexes = {k: extract_protein_data([v], MAX_NUMBER_OF_COMPONENTS)[0] for k, v in data_set_protein_objects.items()}
len(data_set_protein_objects), len(data_set_res_indexes)

(76632, 76632)

## add coord

In [78]:
# TODO
# we have the residue indexes in data_set_res_indexes
# now we just need to fetch the ca coord of each residue, then average accross the inner lists 

## save data to target